In [1]:
import Dataset_2019_2020 as dataset
from Train import train, plot_train_hist, set_seed_mps, Criterion
from Evaluate_multiclass import evaluate
from torch import nn
import torch
import numpy as np
from Models_lesion import ResNet101Meta_19_20

In [2]:
# Load and Preprocess Data
set_seed_mps(42)
train_transform, val_transform, test_transform = dataset.transforms(img_size=224)

train_ds = dataset.cross_MelanomaDS_19_20.train(train_transform)
val_ds   = dataset.cross_MelanomaDS_19_20.val(val_transform)
test_ds  = dataset.cross_MelanomaDS_19_20.test(test_transform)
full_ds = dataset.cross_MelanomaDS_19_20.full()
sampler, weights = dataset.multilabel_sampler(train_ds.df, train_ds.label_cols)

train_loader = dataset.DataLoader(train_ds, sampler=sampler, batch_size=64, shuffle=False, num_workers=4)
val_loader = dataset.DataLoader(val_ds, batch_size=64, num_workers=4)
test_loader = dataset.DataLoader(test_ds, batch_size=64, num_workers=4)

In [25]:
# # Check multiclass sampler
# for i,  (x, y , m ) in enumerate(train_loader):
#     Y.append(y)
#     if i == 30:
#         break
# torch.concat(Y).mean(dim = 0)

tensor([0.1259, 0.1075, 0.1122, 0.1084, 0.1084, 0.1278, 0.1127, 0.0975, 0.0994])

In [12]:
# Set up Model with cross entropy
net = ResNet101Meta_19_20()
net.to('mps')
class_weights = torch.tensor(
    [1, 1, 1,1, 1, 1, 1,1,1],
    device='mps',
    dtype=torch.float
)
criterion = Criterion('one-hot', weights = class_weights)

opt = torch.optim.AdamW([
    {"params": net.backbone.layer4.parameters(), "lr": 1e-5},
    {"params": net.head.parameters(), "lr": 1e-4}
])

# Train only the last layers (freeze the backbone)
for p in net.backbone.parameters():
    p.requires_grad = False
for p in net.backbone.layer4.parameters():
    p.requires_grad = True
for p in net.head.parameters():
    p.requires_grad = True

hist_train_loss_res, hist_val_loss_res, hist_val_acc_res = train(net, train_loader, val_loader, criterion, opt, epochs = 1, best_model_name = "Resnet101_2019-20_cross_weight1.pt")

100%|██████████| 621/621 [17:07<00:00,  1.66s/it]


Saved new best model at epoch 1 with AUC=0.7447
Epoch 1: Val ACC = 0.6973098953005293 Loss Ratio=0.5973, Val MEL AUC=0.7447, Val MEL Pos Acc=0.9210
Best validation pos_acc: 0.9210


In [11]:
# Load Saved Model
state_dict = torch.load("Resnet101_2019-20_cross_weight1.pt", map_location='mps')
net.load_state_dict(state_dict)

acc, per_class_acc, auc_per_class, ys, ps  = evaluate(net, test_loader, device = 'mps', use_meta= True, threshold=0.5)

100%|██████████| 146/146 [03:19<00:00,  1.37s/it]

MEL:
[[2708 6036]
 [  22  518]]

NV:
[[2407 4445]
 [ 172 2260]]

BKL:
[[4266 4714]
 [  27  277]]

DF:
[[8668  595]
 [   3   18]]

VASC:
[[8967  286]
 [   3   28]]

SCC:
[[8467  760]
 [   3   54]]

AK:
[[8494  693]
 [   7   90]]

BCC:
[[8055  879]
 [   9  341]]

UNK:
[[2077 1755]
 [ 161 5291]]

MEL: AUC = 0.7824  ACC = 0.3475
NV: AUC = 0.8415  ACC = 0.5027
BKL: AUC = 0.8252  ACC = 0.4893
DF: AUC = 0.9708  ACC = 0.9356
VASC: AUC = 0.9850  ACC = 0.9689
SCC: AUC = 0.9781  ACC = 0.9178
AK: AUC = 0.9710  ACC = 0.9246
BCC: AUC = 0.9784  ACC = 0.9044
UNK: AUC = 0.9341  ACC = 0.7936


In [ ]:
# np.savez(
#     "Training_metrics_2019-2020.npz",
#     # EfficientNet2
#     hist_train_loss_eff2=hist_train_loss_res,
#     hist_val_loss_eff2=hist_val_loss_res,
#     hist_val_acc_eff2=hist_val_acc_res)
# data = np.load("Training_metrics_2019-2020.npz", allow_pickle=True)
# plot_train_hist(data['hist_train_loss_eff2'], data['hist_val_loss_eff2'], data['hist_val_acc_eff2'], epochs = 2)
# np.savez(
#     "Evaluation_metrics_2019-2020.npz",
#     acc = acc, per_class_acc = per_class_acc, auc_per_class = auc_per_class, ys = ys, ps = ps)

# true_class = ys.argmax(axis=1)
# pred_class = ps.argmax(axis=1)
# plot_confusion_matrix(ps, preds)